# RWKV

> 「変わることがなければ成長することもない  
> 成長することがなければ真に生きていない」  
> ビル・ゲイツ

## RWKVの概要

OpenAIが開発するChatGPTにおけるGPT4は強力であり、同様のモデルが様々登場しつつあるが、モデル動作に必要な計算資源も膨大であり、簡単に自宅やColabで動作させることはできないが、RWKVはこの問題を解決するのではないかと期待されている

言語モデルは、LSTMなどRNNベースのモデルが利用されてきたが、Transformerの登場により状況が一変した
- Transformerにより並列処理が可能となり実行速度が向上した
- しかしながら、必要となる計算資源が膨大になった


## RNNとTransformerの違いを再確認

Recurrent Neural Networks (RNN)とTransformerの違いを再確認する

ある文章があり、その単語のトークン列が$F[0], F[1], ... F[n]$とする

- Transformer
  - Attention Weightを用いて$F[0] ... F[n-1]$の単語の依存関係から$F[n]$の単語を生成する
  - 文章全体の状態を保持して学習し、残差接続(residual connection)、Dropout、LayerNormなどを利用することで、層数を増やしても安定して学習を進めることができる
  - Self-Attentionは離れた位置の依存関係を学習できるが、シーケンス内のすべての要素と他のすべての要素との依存関係を計算するため、計算量とメモリ使用量がシーケンス長(トークン数)の2乗つまり、O(n^2)で大きくなるという欠点がある
  - 一方で計算を並列化することができるため、計算進度を高めることができる

- RNN
  - $F[n-1]$の単語から$F[n]$の単語を生成というプロセスを繰り返して学習
  - 単一状態を保持し、これを繰り返し適用するため、離れた位置ほど単語の依存関係が失われていく
      - 計算を繰り返すため、勾配爆発や勾配消失を発生ひやすい
  - メモリと計算量は文章長に対して線形にスケールする
  - 並列化と拡張性の制限からtransformerと同等の性能を達成することが困難


# RWKV(Receptance Weighted Key Value)とは?

transformerの効率的な並列学習とRNNの効率的な推論の両方を兼ね備えたモデル

名前は、利用する4つのパラメータ$R, W, K, V$に由来する


## RWKVの特徴

次のように纏めることができる

- RNNベースのモデルを用いて、推論の高速化と省メモリ化を実現
  - 推論時のメモリ使用率は最も大きい14Bモデルでも3GB程度
- 数百億のパラメータまでスケールする(これが限界かどうかは不明であるが、スケール則は有効と思われる)、非Transformerアーキテクチャ
  - この規模はRNNベースのモデルでは達成できなかった
- 同一サイズのTransformerと同等の性能を発揮
- 学習時はTransformerと同様の動作であるため、並列化が容易
- 文章長が理論上無限となる(学習時の文章長に依存し、実際は1024程度)
- AttentionではなくRWKVモデルを利用する

RWKVモデルは https://github.com/BlinkDL/ において公開されており、RWKV1からRWKV4までの4つのモデルが存在

ここでは、RWKV2を用いるが、このモデルは推論にRNNモード、学習にTransformerモードを利用する

## TransformerとRWKVの違い

構造の違いを図を用いて表すと次の通り

- RWKVは、特徴的なTime-mixing blockとChannel-mixing blockが存在
- Transformerと異なり、encoder-decoderモデルではない
- 全体の構造はTransformerと類似する
  - Time-mixing blockとmultihead attentionというブロックが同様の配置・連結された構造を持つが中身が異なる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/transformer-rwkv.png" width=700>


# RWKVの詳細

RWKVを特長づけるブロックとしてTime-mixing blockとChannel-mixing blockがある

RWKVという名称の由来にもなっている4つのパラメータはTime-mixingブロックとChannel-mixingブロックで使用され、次の意味を持つ
- R:過去の情報の受容度を表現するReceptanceベクトル
- W:位置の重み減衰ベクトル。訓練可能なモデルパラメータ
- K:一般的なAttention Mechanismと同様のK(Key)ベクトル
- V:一般的なAttention Mechanismと同様のV(Value)ベクトル


## TransformerからRWKVへ

### Transformer

$$Attention(Q,K,V)=softmax(\frac{QK^T}{\sqrt{d_k}})V$$

ここでQ、K、Vはクエリ(Query)、キー(Key)、値(Value)を示し、キーの次元数$d_k$，シーケンス長(トークン数)$N$、次元数$d_v$である

$F[t+1]$の予測に、$F[0]...F[t]$の文章と、現在の単語$x_t$と$F[t]$をそれぞれ比較して文脈全体の依存関係を考慮する
- 具体的には$Qx_t$と$K(F[0]...F[t])$の全ての単語の内積によりAttention Weightを求め、前の各状態$F[i]$と比較して類似度を求める

すべてのQueryベクトルとKeyベクトルの内積を計算するため、計算量が$n^2$となる


### An Attention Free Transformer (AFT)

Self-Attentionの代わりに普通の全結合層(Fully Connected layer)を利用したモデル
- 計算コストを大幅に削減
- 条件によって、Transformerと同等またはそれ以上のパフォーマンスを達成

AFTは次のように求めることができる

$$Attn^+(W,K,V)_t=\frac{\sum^t_{i=1}e^{\omega t,i+k_i}v_i}{\sum^t_{i=1}e^{\omega t,i+k_i}}$$

ここで$\omega_{t,i} \in R^{T\times T}$は、学習した関係における位置バイアスを表す

また、厳密ではないがRWKVを数的に表現すると、

$$v_{i+1} = sigmoid(Rx[t])\cdot \sum^t_{i=0}{Attn^+(W,K,V)_t}$$

となるため、一つ先($t+1$番目)の単語を予測する場合は、$t$番目の単語の予測に$sigmoid(R*F[t])$、$t～0$番目の単語の予測に$v_i$と$\omega_{t,i+k_i}$の2つを用いる

- なお、活性化関数は$sigmoid$である必要はないが、$sigmoid$の性能が高いという評価結果がある
  - この$sigmoid$の項は正規化せずreceptanceと呼ぶ
- MultiHeadAttentionは存在せず、各単語に$0～t$番目の単語との依存関係を求める必要ない
  - $e^{\omega t,i+k_i}v_i$の項が依存関係を表現する

この式により、RNNと同様再帰的に適用することで、$t-1$番目から、$t$番目を予測できる
- 一つ前の$v_{i-1}$に対して$e^\omega$を掛け合わせ、$v_i$に関する項を足し合わせればよい
- この計算量の増大が、高々$O(n)$として表現できることから、計算コストを削減できる


## Time-mixing block



### Time-mixing blockの動作内容

RWKVの時刻$t$における各ベクトル・パラメータであるReceptanceベクトル($r_t$)、Key($k_t$)、Value($v_t$)は、次の式で表すことができる

$$r_t=W_r\cdot(\mu_rx_t+(1−\mu_r)x_{t−1}) \\
k_t=W_k\cdot(\mu_kx_t+(1−\mu_k)x_{t−1}) \\
v_t=W_v\cdot(\mu_vx_t+(1−\mu_v)x_{t−1})$$

これらの値は時刻$t$での更新割合を制御するパラメータであり、0と1の間の値をとる
- $\mu_r$​が大きいほど新しい入力$x_t$​の影響が強く、小さいほど過去の状態$x_{t−1}$​の影響が強くなる

$mu_r$倍の新しい入力(x_t)と$1-mu_r$倍の和であることから、新しい入力と過去の状態との間で線形補間を行う式
- 従って、これを繰り返すことで再帰的に更新できる

これらを使った演算の詳細は次の通りであり、An Attention Free Transformer (AFT）と類似している

$$wk_vt=\frac{\sum_{i=1}^{t−1}e^{−(t−1−i)w+k_i}v_i+e^{u+k_t}v_t}{\sum_{i=1}^{t−1}e^{−(t−1−i)w+k_i}+e^{u+k_t}}$$
$$o_t=W_o\cdot(\sigma(r_t)\odot wk_vt)$$

$wkv_t$​は、$Attn(Q,K,V)$と同様の役割を担っているが、$Q，K，V$の各要素はスカラーのため計算コストが小さい

- 直感的には、時間$t$が増加するにつれてベクトル$o_t$​は長い履歴に依存することを示しているといえる
- ターゲットポジション$t$に対して、RWKVは位置間隔$[1, t]$での加重平均とレセプタンス$sigmoid(R)$と乗算している
  - 与えられたタイムステップ内で乗算され、異なるタイムステップで合計される
- 標準的なトランスフォーマーは全てのトークンのペア間でアテンションを計算するが、AFTは過去の時間ステップ全てにわたる加算の形でアテンションを計算するため、計算とメモリ利用効率が向上している

### 時間減衰率(Time-Weighting)


RWKVはAFTに倣い、$\omega_{t,i}$​をチャネルごとの時間減衰ベクトルとして定義している

$$\omega_{t,i} = -t(t-i)\omega$$

このモデルではTime-MixingおよびTime-Weightingというパラメータを導入している
- Time-Weightingを距離によるAttentionと呼ぶ

Wはヘッド数、block_size(0～現在の時刻)、block_sizeの3次元で構成され、attに$\mathbb{W}$を要素ごとかけた値をdropoutしている

$\mathbb{W}$は比較的小さいパラメータであり、このようなパラメ―タで離れた単語の依存関係を学習可能とする

これは、異なる距離のトークン($W[:, block_size, block_size]$)が現在の単語attに与える影響は異なり、特に文章の後半は長い距離の$\mathbb{W}$を用いる必要があるが、最初は履歴が小さく必要がない

- SelfAttentionはそのような考慮がない

実際には$\mathbb{W}$は巡回行列でありバイアスを加算する必要があり、Time Weightingの導入によりPositionalEncodingが不要になる

### Time-mixing blockの実装

下記コードの関数`RWKV_TimeMix`の概要は次の通り
- 時刻$t$における入力の更新割合$µ_r$​はself.time_mixで表される
- また時刻$t−1$の入力xはnn.ZeroPad2dを用いて表されている
- これらを用いると入力$x$は、`x = x * self.time_mix + self.time_shift(x) * (1 - self.time_mix)`と実装され、時間的にシフトした部分すなわち過去の情報と、シフトしていない部分すなわち現在の情報を適切な比率で混合した値へと変換している


## Channel-mixing block

channel-mixing blockは次の式で表される

$$r_t=W_r\cdot(\mu_rx_t+(1−\mu_r)x_{t−1})$$
$$k_t=W_k\cdot(\mu_kx_t+(1−\mu_k)x_{t−1})$$
$$o_t=\sigma(r_t)\odot(W_v\cdot max(k_t,0)^2)$$

Time-mixing blockは異なる時間ステップのトークン間の相互作用を管理するのに対して、Channel-mixing blockは同じ時間ステップ内の異なるチャンネル（または特徴）間の相互作用を管理する

Channel-mixing blockは、全結合層や畳み込み層と同様の働きを持つ
- $\sigma$はsquared ReLUを使用し、忘却ゲートの役割を果たす

Channel-mixing blockの実装は、Time-mixing blockが理解できれば問題なく理解であろう

# RWKVの特徴

RWKVは学習時time-parallel mode (時間パラレルモード)が使用され、推論時（デコード時）はtime-sequential mode (時間シーケンシャルモード)が使用される

<img src="http://class.west.sd.keio.ac.jp/dataai/text/rwkv.png" width=500>


## 時間パラレルモード

学習時のモード

時間パラレルモードは言葉の通り、時刻に関連する演算を並列して行う

Time-mixing blockで説明したように新しい入力$x_t$と過去の状態との間で線形補間を行う
- 結果として、各タイムステップの計算が他のタイムステップの計算と独立に実行できる
- RNNとは異なり並列処理が可能

## 時間シーケンシャルモード

推論時のモード

学習時とは異なり、推論時にはRNNのような順次的なデコーディングを行う

- このときRWKVはRNNと同様の構造を活用して動作する
  - これを時間シーケンシャルモードと呼ぶ
  - 各ステップの出力が次のステップの入力として用いられる
  - RNN同様、は出力トークンを一度に1つずつ生成し、あるトークンの生成は前のすべてのトークンの生成が完了した後に行わる

RWKVはシーケンスの長さに関係なく一定の速度とメモリフットプリントを維持しする
- 長いシーケンスを効率的に処理できる
- 一方で、アテンションメカニズムを使用しているため、シーケンスの長さに比例してキャッシュの使用量が増加する

以上からRWKVではTransfromerでは実現できなかった効率的なデコーディングが可能となる

# RWKVの学習

実際に学習を行うことができる。ただし、学習にGoogle Colaboratory ProでA100を利用した場合でも10時間強程度必要となるため、基本的に放置になるのと同時に、それなりの課金が必要となる

従って、ここでは、そのリンクのみ示すとして、実際に実行することは避ける



In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import os
if not os.path.exists('cuda.tgz'):
  !gdown "10TVle71vI4B7bB-pPYLEaog4ut74sG6q" -O cuda.tgz
  !tar xzf cuda.tgz
if not os.path.exists('enwik8.tgz'):
  !gdown "10TN_ZnQ3G84vhzoR-AiVfekt4SJMKVW5" -O enwik8.tgz
  !tar xzf enwik8.tgz

Downloading...
From: https://drive.google.com/uc?id=10TVle71vI4B7bB-pPYLEaog4ut74sG6q
To: /content/cuda.tgz
100% 1.49k/1.49k [00:00<00:00, 7.93MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=10TN_ZnQ3G84vhzoR-AiVfekt4SJMKVW5
From (redirected): https://drive.google.com/uc?id=10TN_ZnQ3G84vhzoR-AiVfekt4SJMKVW5&confirm=t&uuid=e6502324-26a1-4e5e-92c4-923086aeec3e
To: /content/enwik8.tgz
100% 35.2M/35.2M [00:00<00:00, 38.8MB/s]


In [3]:
!pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.0 MB/s eta 0:00:00


In [4]:
import json
import random
import time
import math
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.cpp_extension import load
from torch.utils.data import Dataset
import math
import logging
import datetime
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True

### Step 1: set training data ##########################################################################

datafile = "enwik8"
datafile_encoding = 'utf-8'
# datafile_encoding = 'utf-16le'

### Step 2: set model size #############################################################################

ctx_len = 1024        # ===> increase T_MAX in model.py if your ctx_len > 1024
n_layer = 6
n_embd = 512

# 'RWKV' (better for char-level English) or 'RWKV-ffnPre' (better in some cases)
model_type = 'RWKV'

### Step 3: set batch size #############################################################################

# ===> batch_size must be divisible by B_GROUP_FORWARD and B_GROUP_BACKWARD in model.py
# For example, if your batch_size = 20, you can set B_GROUP_FORWARD = 4, B_GROUP_BACKWARD = 2
# If you see "CUDA out of memory", reduce it. Use GPU-Z to find the highest value for your VRAM.
batch_size = 12

### Step 4: set learning rate, training mini-epochs #######################################################

lr_init = 6e-4
lr_final = 1e-5
# the mini-epoch is very short and of fixed length (ctx_len * epoch_length_fixed tokens)
n_epoch = 500
# 0 = never, 1 = every mini-epoch, 2 = every two mini-epochs, etc.
epoch_save_frequency = 30
epoch_save_path = 'trained-'

epoch_length_fixed = 10000

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


In [5]:
########################################################################################################
# CUDA Kernel
########################################################################################################

T_MAX = 1024          # increase this if your ctx_len > 1024
B_GROUP_FORWARD = 4   # set to 8 for best performance
B_GROUP_BACKWARD = 2  # set to 2 for best performance

timex_cuda = load(name="timex", sources=["cuda/timex_op.cpp", "cuda/timex_cuda.cu"],
                  verbose=True, extra_cuda_cflags=['--use_fast_math', '--extra-device-vectorization', f'-DTmax={T_MAX}', f'-DBF={B_GROUP_FORWARD}', f'-DBB={B_GROUP_BACKWARD}'])


class TimeX(torch.autograd.Function):
    @staticmethod
    def forward(ctx, w, k, B, C, T, eps):
        ctx.B = B
        ctx.C = C
        ctx.T = T
        assert ctx.T % 4 == 0 and ctx.T <= T_MAX and ctx.B % B_GROUP_FORWARD == 0 and ctx.B % B_GROUP_BACKWARD == 0
        w = w.contiguous()
        k = k.contiguous()
        ctx.save_for_backward(w, k)
        wk = torch.empty((B, C, T), device='cuda',
                         memory_format=torch.contiguous_format)
        timex_cuda.forward(w, k, wk, eps, B, C, T)
        return wk

    @staticmethod
    def backward(ctx, gwk):
        assert ctx.T % 4 == 0 and ctx.T <= T_MAX and ctx.B % B_GROUP_FORWARD == 0 and ctx.B % B_GROUP_BACKWARD == 0
        w, k = ctx.saved_tensors
        gw = torch.empty((ctx.B, ctx.C, ctx.T), device='cuda',
                         memory_format=torch.contiguous_format)
        gk = torch.empty((ctx.B, ctx.C, ctx.T), device='cuda',
                         memory_format=torch.contiguous_format)
        timex_cuda.backward(w, k, gwk.contiguous(), gw,
                            gk, ctx.B, ctx.C, ctx.T)
        return (gw.sum(dim=0), gk, None, None, None, None)

In [6]:
########################################################################################################
# RWKV: RWKV Time-mix + RWKV Channel-mix
########################################################################################################


RWKV_K_CLAMP = 60  # e^60 = 1e26
RWKV_K_EPS = 1e-16
RWKV_HEAD_QK_DIM = 256


def RWKV_Init(module, config):  # fancy initialization of all lin & emb layer in the module
    for m in module.modules():
        if not isinstance(m, (nn.Linear, nn.Embedding)):
            continue
        with torch.no_grad():
            name = '[unknown weight]'
            for name, parameter in module.named_parameters():  # find the name of the weight
                if id(m.weight) == id(parameter):
                    break

            shape = m.weight.data.shape
            gain = 1.0
            scale = 1.0  # extra scale for gain

            if isinstance(m, nn.Embedding):
                gain = math.sqrt(max(shape[0], shape[1]))
                if shape[0] == config.vocab_size and shape[1] == config.n_embd:  # token emb?
                    scale = 1e-4
                else:
                    scale = 0

            if isinstance(m, nn.Linear):
                if m.bias is not None:
                    m.bias.data.zero_()
                if shape[0] > shape[1]:
                    gain = math.sqrt(shape[0] / shape[1])
                if shape[0] == config.vocab_size and shape[1] == config.n_embd:  # final projection?
                    scale = 0.5

            if hasattr(m, 'scale_init'):
                scale = m.scale_init

            # print(str(shape[0]).ljust(5), str(shape[1]).ljust(5), f'{round(scale,2):g}'.ljust(4), name)

            gain *= scale
            if scale == -999:
                nn.init.eye_(m.weight)
            elif gain == 0:
                # zero init is great for some RWKV matrices
                nn.init.zeros_(m.weight)
            elif gain > 0:
                nn.init.orthogonal_(m.weight, gain=gain)
            else:
                nn.init.normal_(m.weight, mean=0.0, std=-scale)

class RWKV_TimeMix(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.layer_id = layer_id
        self.ctx_len = config.ctx_len
        self.n_embd = config.n_embd

        attn_sz = config.n_embd

        ############# fancy init of time_w curves ###################################
        f1_begin = 3.0
        f1_end = 1.2
        f2_begin = 0.65
        f2_end = 0.4
        with torch.no_grad():  # initial time_w curves for better convergence
            decay_speed = torch.ones(attn_sz, 1)
            first_sa_layer_id = 1
            for h in range(attn_sz):
                f1 = f1_begin + (layer_id-first_sa_layer_id) / \
                    (config.n_layer-1-first_sa_layer_id) * (f1_end - f1_begin)
                f2 = f2_begin + (layer_id-first_sa_layer_id) / \
                    (config.n_layer-1-first_sa_layer_id) * (f2_end - f2_begin)
                if layer_id == first_sa_layer_id:
                    f1 += 0.5
                if layer_id == config.n_layer-2:
                    f2 = 0.4
                if layer_id == config.n_layer-1:
                    f2 = 0.37
                decay_speed[h][0] = math.pow(f2, h / (attn_sz-1) * 7) * f1
        self.time_decay = nn.Parameter(torch.log(decay_speed)) # will use exp(self.time_decay) to ensure time_decay > 0
        self.time_curve = torch.tensor(
            [-(config.ctx_len - 2 - i) for i in range(config.ctx_len-1)]).unsqueeze(0)
        self.time_curve = self.time_curve.to('cuda')
        self.time_first = nn.Parameter(torch.ones(attn_sz, 1) * math.log(0.3))
        #############################################################################

        self.time_shift = nn.ZeroPad2d((0, 0, 1, -1))
        with torch.no_grad():  # init to "shift half of the channels"
            ww = torch.ones(1, 1, config.n_embd)
            for i in range(config.n_embd // 2):
                ww[0, 0, i] = 0
        self.time_mix = nn.Parameter(ww)

        self.key = nn.Linear(config.n_embd, attn_sz, bias=False)
        self.value = nn.Linear(config.n_embd, attn_sz, bias=False)
        self.receptance = nn.Linear(config.n_embd, attn_sz, bias=False)

        self.output = nn.Linear(attn_sz, config.n_embd, bias=False)

        self.key.scale_init = 0
        self.receptance.scale_init = 0
        self.output.scale_init = 0

    def forward(self, x):
        B, T, C = x.size()

        x = x * self.time_mix + self.time_shift(x) * (1 - self.time_mix)

        k = self.key(x).transpose(-1, -2)
        v = self.value(x).transpose(-1, -2)
        r = self.receptance(x)

        # RWKV_K_CLAMP can be removed if the CUDA kernel substracts the correct k_max for each k (I will do this later)
        k = torch.clamp(k, max=RWKV_K_CLAMP)
        k = torch.exp(k)
        kv = k * v

        self.time_w = torch.cat(
            [torch.exp(self.time_decay) * self.time_curve, self.time_first], dim=-1)
        w = torch.exp(self.time_w)

        wkv = TimeX.apply(w, kv, B, C, T, 0)
        # RWKV_K_EPS can be removed if the CUDA kernel sets 0/0 = 0 (I will do this later)
        wk = TimeX.apply(w, k, B, C, T, RWKV_K_EPS)

        rwkv = torch.sigmoid(r) * (wkv / wk).transpose(-1, -2)
        rwkv = self.output(rwkv)
        return rwkv


class RWKV_ChannelMix(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.layer_id = layer_id

        self.time_shift = nn.ZeroPad2d((0, 0, 1, -1))

        with torch.no_grad():  # init to "shift half of the channels"
            x = torch.ones(1, 1, config.n_embd)
            for i in range(config.n_embd // 2):
                x[0, 0, i] = 0
        self.time_mix = nn.Parameter(x)

        hidden_sz = 4 * config.n_embd
        self.key = nn.Linear(config.n_embd, hidden_sz, bias=False)
        self.receptance = nn.Linear(config.n_embd, config.n_embd, bias=False)
        self.value = nn.Linear(hidden_sz, config.n_embd, bias=False)

        self.value.scale_init = 0
        self.receptance.scale_init = 0

    def forward(self, x):
        x = x * self.time_mix + self.time_shift(x) * (1 - self.time_mix)

        k = self.key(x)
        k = torch.square(torch.relu(k))
        kv = self.value(k)

        rkv = torch.sigmoid(self.receptance(x)) * kv
        return rkv

In [7]:
########################################################################################################
# The GPT Model with our blocks
########################################################################################################

class GPTConfig:
    def __init__(self, vocab_size, ctx_len, **kwargs):
        self.vocab_size = vocab_size
        self.ctx_len = ctx_len
        for k, v in kwargs.items():
            setattr(self, k, v)

class Block(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.config = config
        self.layer_id = layer_id

        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)

        if self.layer_id == 0 and self.config.model_type == 'RWKV-ffnPre':
            self.ffnPre = RWKV_ChannelMix(config, layer_id+1000)
        else:
            self.att = RWKV_TimeMix(config, layer_id)

        self.ffn = RWKV_ChannelMix(config, layer_id)

    def forward(self, x):
        x = self.ln1(x)
        if self.layer_id == 0 and self.config.model_type == 'RWKV-ffnPre':
            x = x + self.ffnPre(x)  # better in some cases
        else:
            x = x + self.att(x)
        x = self.ln2(x)
        x = x + self.ffn(x)
        return x

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.step = 0
        self.config = config

        self.emb = nn.Embedding(config.vocab_size, config.n_embd)

        self.blocks = nn.Sequential(*[Block(config, i)
                                    for i in range(config.n_layer)])

        self.ln_out = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.head_q = nn.Linear(config.n_embd, RWKV_HEAD_QK_DIM, bias=False)
        self.head_q.scale_init = 0
        self.head_k = nn.Linear(config.n_embd, RWKV_HEAD_QK_DIM, bias=False)
        self.head_k.scale_init = 0.1
        self.register_buffer("copy_mask", torch.tril(
            torch.ones(config.ctx_len, config.ctx_len)))

        self.ctx_len = config.ctx_len

        RWKV_Init(self, config)

        logger.info("number of parameters: %e", sum(p.numel()
                    for p in self.parameters()))

    def get_ctx_len(self):
        return self.ctx_len

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear)):
            module.weight.data.normal_(mean=0.0, std=0.01)
        if isinstance(module, (nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=1e-5)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()

    def configure_optimizers(self, train_config):
        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()

        for mn, m in self.named_modules():  # here we disable weight_decay
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn  # full param name
                no_decay.add(fpn)

        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(
            inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
            % (str(param_dict.keys() - union_params), )

        optim_groups = [
            {"params": [param_dict[pn]
                        for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]

        optimizer = torch.optim.Adam(
            optim_groups, lr=train_config.learning_rate, betas=train_config.betas, eps=train_config.eps)

        return optimizer

    def forward(self, idx, targets=None):
        self.step += 1
        B, T = idx.size()
        assert T <= self.ctx_len, "Cannot forward, because len(input) > model ctx_len."
        x = self.emb(idx)

        x = self.blocks(x)

        x = self.ln_out(x)

        q = self.head_q(x)[:, :T, :]
        k = self.head_k(x)[:, :T, :]
        c = (q @ k.transpose(-2, -1)) * (1.0 / RWKV_HEAD_QK_DIM)
        c = c.masked_fill(self.copy_mask[:T, :T] == 0, 0)

        c = c @ F.one_hot(idx, num_classes=self.config.vocab_size).float()
        x = self.head(x) + c

        loss = None
        if targets is not None:
            loss = F.cross_entropy(x.view(-1, x.size(-1)), targets.view(-1))

        return x, loss

In [8]:
########################################################################################################
# The RWKV v2-RNN Language Model - https://github.com/BlinkDL/RWKV-LM
########################################################################################################

class Dataset(Dataset):
    def __init__(self, data, ctx_len, epoch_length_fixed):
        print('building token list...', end=' ')
        unique = sorted(list(set(data)))
        # print()
        # for u in unique:
        #     print(u, end=' ')
        # print('\n\n')

        xx = 0
        xxObj = {}
        for u in unique:
            xxObj[xx] = u
            xx += 1
        with open('vocab.json', "w", encoding="utf-16") as vocab_file:
            vocab_file.write(json.dumps(xxObj, ensure_ascii=False))

        data_size, vocab_size = len(data), len(unique)
        print('data has %d tokens, %d unique.' % (data_size, vocab_size))
        self.stoi = {ch: i for i, ch in enumerate(unique)}
        self.itos = {i: ch for i, ch in enumerate(unique)}
        self.ctx_len = ctx_len
        self.epoch_length_fixed = epoch_length_fixed
        self.vocab_size = vocab_size
        self.data = data

    def __len__(self):
        return self.epoch_length_fixed

    def __getitem__(self, idx):
        # cheat: pick a random spot in dataset
        i = np.random.randint(0, len(self.data) - (self.ctx_len + 1))
        chunk = self.data[i:i+self.ctx_len+1]
        dix = [self.stoi[s] for s in chunk]
        x = torch.tensor(dix[:-1], dtype=torch.long,
                         device=torch.device('cuda'))
        y = torch.tensor(dix[1:], dtype=torch.long,
                         device=torch.device('cuda'))
        return x, y


class TOKENIZER():
    def __init__(self, WORD_NAME, UNKNOWN_CHAR='\ue083'):
        with open(WORD_NAME + '.json', "r", encoding="utf-16") as result_file:
            self.word_table = json.load(result_file)

        self.vocab_size = len(self.word_table)

        self.stoi = {v: int(k) for k, v in self.word_table.items()}
        self.itos = {int(k): v for k, v in self.word_table.items()}

        self.UNKNOWN_CHAR = self.stoi[UNKNOWN_CHAR]

    def refine_context(self, context):
        context = context.strip().split('\n')
        for c in range(len(context)):
            context[c] = context[c].strip().strip('\u3000').strip('\r')
        context = list(filter(lambda c: c != '', context))
        context = '\n' + ('\n'.join(context)).strip()
        if context == '':
            context = '\n'

        return context

    def sample_logits(self, out, x, ctx_len, temperature=1.0, top_p_usual=None, top_p_newline=None):
        # out[self.UNKNOWN_CHAR] = -float('Inf')

        lastChar = int(x[-1])

        probs = F.softmax(torch.tensor(out), dim=-1)

        if self.itos[lastChar] == '\n':
            top_p = top_p_newline
        else:
            top_p = top_p_usual

        sorted_probs, s_index = torch.sort(probs, descending=True)

        # for j in range(30):
        #     pp = sorted_probs[j].item()
        #     if pp < 0.005:
        #         break
        #     ss = self.itos[int(s_index[j])].replace('\n','_')
        #     print(f'{math.floor(pp*100):>3.0f}{ss}', end='')
        # print('')

        cumulative_probs = torch.cumsum(sorted_probs, dim=-1).numpy()
        cutoff = float(sorted_probs[np.argmax(cumulative_probs > top_p)])

        probs[probs < cutoff] = 0
        # print("[" + str(round(cutoff,4)) + ' ' + str(round(to_float(sum(probs)),3)) + "]", end = "")

        if temperature != 1.0:
            probs = probs.pow(1.0 / temperature)

        return torch.multinomial(probs, num_samples=1)[0]


def to_float(x):
    return x.cpu().detach().numpy().flatten()[0].astype(float)


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [9]:
########################################################################################################
# The RWKV v2-RNN Language Model - https://github.com/BlinkDL/RWKV-LM
########################################################################################################

from torch.utils.data.dataloader import DataLoader
from torch.optim.lr_scheduler import LambdaLR
from torch.nn import functional as F
import torch.nn as nn
import torch.optim as optim
import torch
from tqdm.auto import tqdm
import numpy as np
import logging
import os
import datetime
import sys
import math

# import wandb  # comment this if you don't have wandb
# print('logging to wandb... (comment it if you don\'t have wandb)')

logger = logging.getLogger(__name__)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True

log_file = open("mylog.txt", "a")


class TrainerConfig:
    max_epochs = 10
    batch_size = 64
    learning_rate = 4e-4
    betas = (0.9, 0.99)
    eps = 1e-8
    grad_norm_clip = 1.0
    lr_decay = True  # linear warmup followed by cosine decay
    warmup_tokens = 0
    final_tokens = 0
    epoch_save_frequency = 0
    epoch_save_path = 'trained-'
    num_workers = 0  # for DataLoader

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)


class Trainer:

    def __init__(self, model, train_dataset, test_dataset, config):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = config
        self.avg_loss = -1
        self.steps = 0

        if 'wandb' in sys.modules:
            cfg = model.config
            for k in config.__dict__:
                setattr(cfg, k, config.__dict__[k])  # combine cfg
            wandb.init(project="RWKV-LM", name=self.get_run_name() + '-' +
                       datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S'), config=cfg, save_code=False)

        self.device = 'cpu'
        if torch.cuda.is_available():  # take over whatever gpus are on the system
            self.device = torch.cuda.current_device()

    def get_run_name(self):
        raw_model = self.model.module if hasattr(
            self.model, "module") else self.model
        cfg = raw_model.config
        run_name = str(cfg.vocab_size) + '-' + str(cfg.ctx_len) + '-' + \
            cfg.model_type + '-' + str(cfg.n_layer) + '-' + str(cfg.n_embd)
        return run_name

    def train(self):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = raw_model.configure_optimizers(config)

        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset

            if config.num_workers > 0:
                loader = DataLoader(data, shuffle=False, pin_memory=True,
                                    batch_size=config.batch_size,
                                    num_workers=config.num_workers)
            else:
                loader = DataLoader(data, shuffle=False,
                                    batch_size=config.batch_size,
                                    num_workers=config.num_workers)

            pbar = tqdm(enumerate(loader), total=len(
                loader), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}') if is_train else enumerate(loader)

            for it, (x, y) in pbar:
                x = x.to(self.device)  # place data on the correct device
                y = y.to(self.device)

                with torch.set_grad_enabled(is_train):
                    _, loss = model(x, y)  # forward the model

                if is_train:  # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()

                    if config.grad_norm_clip > 0:
                        torch.nn.utils.clip_grad_norm_(
                            model.parameters(), config.grad_norm_clip)

                    optimizer.step()

                    if config.lr_decay:  # decay the learning rate based on our progress
                        # number of tokens processed this step (i.e. label is not -100)
                        self.tokens += (y >= 0).sum()
                        lr_final_factor = config.lr_final / config.learning_rate
                        if self.tokens < config.warmup_tokens:
                            # linear warmup
                            lr_mult = lr_final_factor + \
                                (1 - lr_final_factor) * float(self.tokens) / \
                                float(config.warmup_tokens)
                            progress = 0
                        else:
                            # cosine learning rate decay
                            progress = float(self.tokens - config.warmup_tokens) / float(
                                max(1, config.final_tokens - config.warmup_tokens))
                            lr_mult = (0.5 + lr_final_factor / 2) + (0.5 - lr_final_factor /
                                                                     2) * math.cos(math.pi * progress)  # better 1.0 ~ 0.1
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    now_loss = loss.item()  # report progress
                    self.lr = lr

                    if 'wandb' in sys.modules:
                        wandb.log({"loss": now_loss},
                                  step=self.steps * self.config.batch_size)
                    self.steps += 1

                    if self.avg_loss < 0:
                        self.avg_loss = now_loss
                    else:
                        factor = 1 / (it + 1)
                        self.avg_loss = self.avg_loss * \
                            (1.0 - factor) + now_loss * factor
                    pbar.set_description(
                        f"mini-epoch {epoch+1} prog {progress*100.0:.2f}% iter {it}: ppl {math.exp(self.avg_loss):.2f} loss {self.avg_loss:.4f} lr {lr:e}")

        self.tokens = 0  # counter used for learning rate decay
        for epoch in range(config.max_epochs):

            run_epoch('train')

            log_file.write(
                f'{epoch+1} {self.avg_loss:.6f} {math.exp(self.avg_loss):.4f} {self.lr:.8f} {datetime.datetime.now()} \n')
            log_file.flush()

            if (self.config.epoch_save_frequency > 0 and epoch % self.config.epoch_save_frequency == 0) or (epoch == config.max_epochs - 1):
                # DataParallel wrappers keep raw model object in .module
                raw_model = self.model.module if hasattr(
                    self.model, "module") else self.model
                torch.save(raw_model.state_dict(),
                           self.config.epoch_save_path + str(epoch+1) + '.pth')

In [ ]:
########################################################################################################

# import src.utils
# src.utils.set_seed(42) # remember to change seed if you load a model

np.set_printoptions(precision=4, suppress=True, linewidth=200)
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO,)

grad_norm_clip = 1.0
warmup_tokens = 0

betas = (0.9, 0.99)
eps = 4e-9

num_workers = 0

########################################################################################################
# Load data
########################################################################################################

print('loading data... ' + datafile)
train_dataset = Dataset(open(
    datafile, "r", encoding=datafile_encoding).read(), ctx_len, epoch_length_fixed)

########################################################################################################
# Train model
########################################################################################################
model = GPT(GPTConfig(train_dataset.vocab_size, train_dataset.ctx_len, model_type=model_type,
                      n_layer=n_layer, n_embd=n_embd)).cuda()

# # # load a trained model. remember to change random seed
# m2 = torch.load('trained-61.pth')
# model.load_state_dict(m2)

print('model', model_type, 'epoch', n_epoch, 'batchsz', batch_size, 'betas',
      betas, 'eps', eps, 'ctx', ctx_len, 'layer', n_layer, 'embd', n_embd, )
tconf = TrainerConfig(model_type=model_type, max_epochs=n_epoch, batch_size=batch_size,
                      learning_rate=lr_init, lr_decay=True, lr_final=lr_final, betas=betas, eps=eps, grad_norm_clip=grad_norm_clip,
                      warmup_tokens=warmup_tokens, final_tokens=n_epoch*len(train_dataset)*ctx_len, num_workers=num_workers, epoch_save_frequency=epoch_save_frequency, epoch_save_path=epoch_save_path)
trainer = Trainer(model, train_dataset, None, tconf)

trainer.train()

savepthname = 'trained-' + str(n_epoch) + '-' + trainer.get_run_name() + \
            '-' + datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S') + '.pth'

torch.save(model.state_dict(), savepthname)

loading data... enwik8
building token list... data has 99621832 tokens, 6064 unique.
model RWKV epoch 500 batchsz 12 betas (0.9, 0.99) eps 4e-09 ctx 1024 layer 6 embd 512


  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/834 [00:00<?, ?it/s]

...

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/834 [00:00<?, ?it/s]

  0%|          | 0/834 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/' + savepthname)

ここまでが上手くいけば、問題なく動作するはずである

計算コストが大きく学習できなかった場合は、あらかじめ準備した学習結果をロードして続きを実行すること

In [ ]:

RWKV_K_CLAMP = 60
RWKV_K_EPS = 1e-16
RWKV_HEAD_QK_DIM = 256

DEBUG_TIME = False   # True False - show trained time-coeffs


class RWKV_RNN():
    def __init__(self, MODEL_NAME, RUN_DEVICE, model_type, n_layer, n_embd, ctx_len):
        self.RUN_DEVICE = RUN_DEVICE
        self.model_type = model_type
        self.n_layer = n_layer
        self.n_embd = n_embd
        self.ctx_len = ctx_len

        self.w = types.SimpleNamespace()

        w = torch.load(MODEL_NAME + '.pth',
                       map_location=torch.device(RUN_DEVICE))
        for x in w.keys():
            if '.time_' in x:
                w[x] = w[x].squeeze()
            if '.time_decay' in x:
                w[x] = torch.exp(-torch.exp(w[x]))
            if '.time_first' in x:
                w[x] = torch.exp(w[x])
            if DEBUG_TIME and '.time_' in x:
                print(x, w[x].squeeze().cpu().numpy())

            xx = x.split('.')
            here = self.w
            for i in range(len(xx)):
                if xx[i].isdigit():
                    ii = int(xx[i])
                    if ii not in here:
                        here[ii] = types.SimpleNamespace()
                    here = here[ii]
                else:
                    if i == len(xx) - 1:
                        setattr(here, xx[i], w[x])
                    elif not hasattr(here, xx[i]):
                        if xx[i+1].isdigit():
                            setattr(here, xx[i], {})
                        else:
                            setattr(here, xx[i], types.SimpleNamespace())
                    here = getattr(here, xx[i])

        self.clear()

    def clear(self):
        self.xx = {}
        self.aa = {}
        self.bb = {}
        self.hk = None

    def save(self, target):
        target.xx = copy.deepcopy(self.xx)
        target.aa = copy.deepcopy(self.aa)
        target.bb = copy.deepcopy(self.bb)
        target.hk = copy.deepcopy(self.hk)

    def load(self, target):
        self.xx = copy.deepcopy(target.xx)
        self.aa = copy.deepcopy(target.aa)
        self.bb = copy.deepcopy(target.bb)
        self.hk = copy.deepcopy(target.hk)

    def LN(self, xx, w):
        return F.layer_norm(xx, (self.n_embd,), weight=w.weight, bias=w.bias)

    def FF(self, xx, w, name):
        if name not in self.xx:
            self.xx[name] = torch.zeros(self.n_embd, device=self.RUN_DEVICE)
        x = xx * w.time_mix + self.xx[name] * (1 - w.time_mix)
        self.xx[name] = xx

        r = torch.sigmoid(w.receptance.weight @ x)
        k = torch.square(torch.relu(w.key.weight @ x))
        kv = w.value.weight @ k

        return r * kv

    def SA(self, xx, w, name):
        if name not in self.xx:
            self.xx[name] = torch.zeros(self.n_embd, device=self.RUN_DEVICE)
            self.aa[name] = torch.zeros(self.n_embd, device=self.RUN_DEVICE)
            self.bb[name] = torch.zeros(self.n_embd, device=self.RUN_DEVICE)
        x = xx * w.time_mix + self.xx[name] * (1 - w.time_mix)
        self.xx[name] = xx

        r = torch.sigmoid(w.receptance.weight @ x)

        k = torch.exp(torch.clamp(w.key.weight @ x, max=RWKV_K_CLAMP))
        v = w.value.weight @ x
        kv = k * v

        a = self.aa[name] + w.time_first * kv
        b = self.bb[name] + w.time_first * k
        self.aa[name] = w.time_decay * self.aa[name] + kv
        self.bb[name] = w.time_decay * self.bb[name] + k

        rwkv = r * a / (b + RWKV_K_EPS)

        return w.output.weight @ rwkv

    def run(self, ctx):
        w = self.w
        x = w.emb.weight[ctx[-1]]

        for i in range(self.n_layer):
            x = self.LN(x, w.blocks[i].ln1)
            if i == 0 and self.model_type == 'RWKV-ffnPre':
                x = x + self.FF(x, w.blocks[i].ffnPre, f'ffnPre.{i}')
            else:
                x = x + self.SA(x, w.blocks[i].att, f'att.{i}')
            x = self.LN(x, w.blocks[i].ln2)
            x = x + self.FF(x, w.blocks[i].ffn, f'ffn.{i}')

        x = self.LN(x, w.ln_out)

        if self.hk == None:
            self.hk = (w.head_k.weight @ x).unsqueeze(0)
        else:
            self.hk = torch.cat(
                [self.hk, (w.head_k.weight @ x).unsqueeze(0)], dim=0)
        if self.hk.shape[0] > self.ctx_len:
            self.hk = self.hk[-self.ctx_len:, :]

        q = w.head_q.weight @ x

        x = w.head.weight @ x
        x = x.cpu().numpy().tolist()

        c = (self.hk @ q) / RWKV_HEAD_QK_DIM
        for i in range(len(c)):
            x[ctx[i]] += c[i]

        return x

GPT-3などに比較すれば、これでもかなり学習時間や計算コストが小さいが、次のように、文章生成を行うことができる点に注目されたい

In [ ]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True
np.set_printoptions(precision=4, suppress=True, linewidth=200)

### Step 1: set model ##################################################################################

ctx_len = 1024
n_layer = 6
n_embd = 512
model_type = 'RWKV'           # 'RWKV' or 'RWKV-ffnPre'

# your trained model
MODEL_NAME = 'trained-v1'
WORD_NAME = 'vocab'           # the .json vocab (generated by train.py

# ########## Uncomment these to test my 27M params enwik8 model ##########
# MODEL_NAME = 'enwik8-ppl1.65-6064-1024-RWKV-6-512-2022-03-25-21-05-13'
# WORD_NAME = 'enwik8-vocab'
# EVAL_DATA = 'enwik8'  # uncomment this for EVAL MODE (no text generation)
# ########################################################################

# --> set UNKNOWN_CHAR to the rarest token in your vocab.json <--
# --> all unknown tokens in your context will be denoted by it <--
UNKNOWN_CHAR = ' '   # here we just set it to [space] for simplicity

RUN_DEVICE = 'cpu'   # 'cpu' (already very fast) or 'cuda'
DEBUG_DEBUG = False  # True False - show softmax output

### Step 2: set context ################################################################################

context = "\nIn the"       # ==> this is your prompt

NUM_TRIALS = 10
LENGTH_PER_TRIAL = 500

TEMPERATURE = 1.0
top_p = 0.7
top_p_newline = 0.9

########################################################################################################

print(f'Loading {MODEL_NAME}...')
model = RWKV_RNN(MODEL_NAME, RUN_DEVICE, model_type, n_layer, n_embd, ctx_len)
tokenizer = TOKENIZER(WORD_NAME, UNKNOWN_CHAR=UNKNOWN_CHAR)

########################################################################################################

if 'EVAL_DATA' in vars() or 'EVAL_DATA' in globals():
    print('Evaluating on ' + EVAL_DATA + ' ...')

    data = open(EVAL_DATA, "r", encoding='utf-8').read()

    loss_table = np.zeros(ctx_len)

    N_SAMPLE = 1000

    for iii in range(N_SAMPLE):
        pos = np.random.randint(0, len(data) - ctx_len-1)
        context = data[pos:pos+ctx_len+1]
        ctx = [tokenizer.stoi.get(s, tokenizer.UNKNOWN_CHAR) for s in context]

        model.clear()
        for i in range(1, ctx_len+1):
            x = ctx[:i]
            out = model.run(x)
            prob = F.softmax(torch.tensor(out), dim=-1)
            loss_table[i-1] += -math.log(prob[ctx[i]])

        print(f'Tested {iii+1} samples: avg_loss over ctx_len =',
              np.mean(loss_table) / (iii+1))

    exit(0)

########################################################################################################

context = tokenizer.refine_context(context)
print('\nYour prompt has ' + str(len(context)) + ' tokens.')
print('\n--> Currently the first run takes a while if your prompt is long, as we are using RNN to process the prompt. This will be much faster in future versions. <--\n')

for TRIAL in range(1 if DEBUG_DEBUG else NUM_TRIALS):
    t_begin = time.time_ns()

    src_len = len(context)
    ctx = [tokenizer.stoi.get(s, tokenizer.UNKNOWN_CHAR) for s in context]
    print(('-' * 30) + context, end='')

    model.clear()
    if TRIAL == 0:
        init_state = types.SimpleNamespace()
        for i in range(src_len):
            x = ctx[:i+1]
            if i == src_len - 1:
                init_state.out = model.run(x)
            else:
                model.run(x)
        model.save(init_state)
    else:
        model.load(init_state)

    for i in range(src_len, src_len + (1 if DEBUG_DEBUG else LENGTH_PER_TRIAL)):
        x = ctx[:i+1]
        x = x[-ctx_len:]

        if i == src_len:
            out = copy.deepcopy(init_state.out)
        else:
            out = model.run(x)
        if DEBUG_DEBUG:
            print('model', np.array(x), '==>', np.array(
                out), np.max(out), np.min(out))

        char = tokenizer.sample_logits(out, x, ctx_len, temperature=TEMPERATURE,
                                       top_p_usual=top_p, top_p_newline=top_p_newline)
        char = char.item()
        print(tokenizer.itos[int(char)], end='', flush=True)
        ctx += [char]
    t_end = time.time_ns()
    print("\n----------", round((t_end - t_begin) / (10 ** 9), 2), end='s ')

課題1

RWKVはなぜ期待されているのか、自分の言葉で纏めなさい

課題2

その他、注目する技術は次々と登場している

何でもよいので、最新の機械学習に関連する技術について調査を行い、簡単に纏めなさい